In [1]:
%run setup
%load_ext autoreload
%autoreload 2

In [2]:
from plotly.offline import iplot
import numpy as np
from nlp_intro.data import create_2d_2class_data
from nlp_intro.logistic_regression import fit
import plotly.graph_objects as go
from nlp_intro.figure_factory import Linear2D2ClassModelPlotter

ImportError: cannot import name 'create_2d_2class_data' from 'nlp_intro.data' (C:\Users\peter\Development\nlp-intro\nlp_intro\data.py)

In [4]:
from plotly import __version__
__version__

'4.0.0'

In [5]:
model = lambda x: 2*x+1

In [6]:
features, labels = create_2d_2class_data((0,1), (0,4), 100, model, .0)

In [10]:
plotter = Linear2D2ClassModelPlotter(features, labels, model, height=700, width=700)

In [11]:
plotter.enable_model_areas()

In [12]:
plotter()

FigureWidget({
    'data': [{'fill': 'toself',
              'fillcolor': 'rgba(255,65,54,0.5)',
             …

In [17]:
x, y, k = fit(features, labels, lr=1, max_iter=10000, verbose=False)
print(f"y = {-x/y:.2f}x + {-k/y:.2f}")

y = 2.00x + 1.07
